# Interactive development workflow
This example illustrates how to develop, run, test(, repeat) a Modelica model in a (local) pandas environment, enabling quick iterations to analyze and tune the model before going to Big Data.

To run the example of the parametric simulation you need to run one of the following sections to build the model executeables.

See https://openmodelica.org/doc/OpenModelicaUsersGuide/v1.11.0/ompython.html

In [ ]:
import pandas as  pd
import os
import sys

In [ ]:
# Finds the package in the repo instead of the installed one
sys.path.insert(0, '../../../src')

In [ ]:
from OMPython import ModelicaSystem
#import OMPython

In [ ]:
import tempfile
import json
import importlib
#from OMPython import ModelicaSystem
from MM import BouncingBall
import DyMat

## Either build model and run simulation with interactive OMC session...
This section demonstrates how to simulate and of the model using the [OMPython API](https://www.openmodelica.org/doc/OpenModelicaUsersGuide/latest/ompython.html) ***with*** a connection to omc enabling the the manipulation of parameters through `setParameters`.

In [ ]:
# mod = BouncingBall.instantiatemodel()  # Does NOT connect to omc
modelname = BouncingBall.modelName
# xmlFilePath = os.path.join(SETUP_DIR, 'MM', 'build', modelname)
mod = ModelicaSystem()
mod.model(
    model_name=modelname,
    model_file=os.path.dirname(BouncingBall.__file__) + '/' + modelname+".mo",
    )

In [ ]:
sim_options_d = mod.getSimulationOptions()
sim_options_d['stopTime'] = 2
mod.setSimulationOptions(sim_options_d)

In [ ]:
mod.getSimulationOptions() setParameters getParameters getSimulationOptions setSimulationOptions

In [ ]:
%%time
resfilename = os.path.join(tempfile.gettempdir(), 'b.mat')
mod.setParameters("e=0.2")  # works in interactive mode only
mod.simulate(resultfile=resfilename, simflags=None)

In [ ]:
# Needs connection to omc
vars = mod.getSolutions()
vars

In [ ]:
%%time
# TODO Loading results (from mat-file) is very slow
data = mod.getSolutions(varList=list(vars), resultfile=resfilename)
ts_df = pd.DataFrame(data).T
ts_df.columns = vars
ts_df

In [ ]:
%matplotlib inline
#%matplotlib notebook
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(figsize=(5,3.5))
ax.plot(ts_df['time'], ts_df['h'])
plt.tight_layout()

## Or build a model without connecting to OMC
This section demonstrate how to build a model using modelica `*.mos` script files ***without*** connection to omc.

In [ ]:
BouncingBall.create_mos_file()  # creates a *.mos file next to the location of BouncingBall

In [ ]:
r = BouncingBall.run_mos_file()  # builds the model executable in the current working directory

In [ ]:
r

## Parametric simulation without connecting to omc
Parameters can be provided to precompiled models as arguments - without connecting or even requiring omc. `instantiatemodel('BouncingBall')` looks for the model (*BouncingBall_init.xml*) in the current working directory (when created by the section above) or in *src/MM/build/BouncingBall* when created via `python setup.py build`.

See also: https://openmodelica.org/doc/OpenModelicaUsersGuide/latest/scripting_api.html#simulation-parameter-sweep

In [ ]:
from MM import BouncingBall

In [ ]:
parameters_var_df = pd.DataFrame(columns=['run_key', 'modifiers'], data=[
    ['r1', {'override_variables': {'e': 0.7,}, 'sim_options_override': {'stopTime': 3.0,}}],
    ['r2', {'override_variables': {'e': 0.5,}, 'sim_options_override': {'stopTime': 3.0,}}],
    ['r3', {'override_variables': {'e': 0.9,}, 'sim_options_override': {'stopTime': 3.0,}}],
])
parameters_var_df['modifiers'] = parameters_var_df['modifiers'].apply(json.dumps)
parameters_var_df

### Sequential execution

In [ ]:
from OMPython import ModelicaSystemRunner


def instantiatemodel(modelName, use_local=True, force_executable_path=None):
    mod = ModelicaSystemRunner(
            modelname=modelName,
            runpath='.',
        )
    return mod

mod = instantiatemodel(modelName='BouncingBall')

In [ ]:
mod = BouncingBall.instantiatemodel('./BouncingBall')

In [ ]:
mod.getOptimizationOptions()

In [ ]:
mod.getParameters()

In [ ]:
from ModelicaRuntimeTools import run_sim_parametric, dymat2pandas, get_sim_dist_func

In [ ]:
%%time
ts_all_df = parameters_var_df.groupby(['run_key']).apply(
        get_sim_dist_func(BouncingBall, 'BouncingBall', res_vars=['h', 'v'],
        modelicaSystemRunner=mod,
        )  # , include_groups=False
    )

In [ ]:
ts_all_df

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
p_df = ts_all_df[[
    'time', 'run_key', 'h'
]].reset_index(drop=True).drop_duplicates(subset=['time', 'run_key'])
p_df.pivot(index='time',columns='run_key', values='h').plot()

### Parallel execution with Spark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ModelicaBouncingBall").getOrCreate()

In [ ]:
parameters_var_sdf = spark.createDataFrame(parameters_var_df)
parameters_var_sdf.show()

In [ ]:
parameters_var_sdf.toPandas()['modifiers'].iloc[0]

### Parallel execution

In [ ]:
from ModelicaRuntimeTools import addpymodules

In [ ]:
sc = spark.sparkContext

In [ ]:
addpymodules([os.path.dirname(DyMat.__file__)], 'mdymat.zip', sc=sc, dironly=True)
addpymodules([os.path.dirname(OMPython.__file__)], 'mOMPython.zip', sc=sc, dironly=True)

In [ ]:
# Run the model wrapper module in the current namespace
modelwrapper_pyfile = BouncingBall.__file__

In [ ]:
%run -i $modelwrapper_pyfile

In [ ]:
instantiatemodel

In [ ]:
from pyspark.sql import types as T, functions as F
res_schema = T.StructType([
    T.StructField("time", T.DoubleType(), True),
    T.StructField("h", T.DoubleType(), True),
    T.StructField("v", T.DoubleType(), True),
    T.StructField("run_key", T.StringType(), True),
])
# Running the parametric simulation
ts_sim_sdf = parameters_var_sdf.groupby(['run_key']).applyInPandas(
        get_sim_dist_func(BouncingBall, 'BouncingBall', res_vars=['h', 'v'], use_local=False), schema=res_schema,
    ).cache()

In [ ]:
ts_sim_sdf.show()

In [ ]:
for n, run_key in enumerate(parameters_var_df['run_key'].unique()):
    ts_run_sim_pdf = ts_sim_sdf.where(F.col('run_key') == run_key).toPandas()
    plt.plot(ts_run_sim_pdf['time'], ts_run_sim_pdf['h'])
plt.show()

## Testing the build script

In [ ]:
from ModelicaBuildTools import build_script

In [ ]:
modelname = "BouncingBall"

r = build_script(BouncingBall, modelname, cwd=os.getcwd())
# modelname = "EDrives.Examples.DCDC.DC_Drive_Switching"
# xmlFilePath = os.path.join(SETUP_DIR, 'MM', 'build', modelname)

# mod = ModelicaSystem(
#         fileName=os.path.join(os.path.dirname(EDrives.__file__), 'EDrives', 'package.mo'),
#         modelName=modelname, lmodel=['Modelica'],
#     )

In [ ]:
r